# Reproduce Figures (Summary Tables)

This notebook reproduces the key calculations used in the report/presentation from the dataset in `data/tiktok_travel_visa_dataset.csv`.

It outputs summary tables to `finalreport/summary_tables/` that you can use to recreate charts (bar charts, line charts, etc.) in Excel or in Python later.

## Outputs
- `hook_distribution.csv`
- `engagement_by_hook.csv`
- `duration_band_performance.csv`
- `category_engagement.csv`
- `trending_sound_lift.csv`


In [11]:
import csv
import os
from pathlib import Path
from collections import defaultdict
from dataclasses import dataclass
from typing import Dict, Iterable, List, Tuple

# Optional Excel export (if openpyxl is installed)
try:
    from openpyxl import Workbook
except Exception:
    Workbook = None


def find_project_root(start: Path) -> Path:
    """Find the project root by walking upwards until the data file exists."""
    target_rel = Path('data') / 'tiktok_travel_visa_dataset.csv'
    p = start.resolve()
    for _ in range(10):
        if (p / target_rel).exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    raise FileNotFoundError(
        f"Could not find project root containing {target_rel}. "
        f"Current working directory: {Path.cwd()}"
    )


PROJECT_ROOT = find_project_root(Path.cwd())
DATASET_CSV = PROJECT_ROOT / 'data' / 'tiktok_travel_visa_dataset.csv'
OUTPUT_DIR = PROJECT_ROOT / 'finalreport' / 'summary_tables'
OUTPUT_XLSX = OUTPUT_DIR / 'reproducible_summary_tables.xlsx'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print('Project root:', PROJECT_ROOT)
print('Dataset:', DATASET_CSV)
print('Output dir:', OUTPUT_DIR)
print('Excel export:', 'enabled' if Workbook is not None else 'disabled (install openpyxl)')

Project root: /Users/richarddanquah/Desktop/Tuc/data analysis/Tiktok-Data-Analysis-Project
Dataset: /Users/richarddanquah/Desktop/Tuc/data analysis/Tiktok-Data-Analysis-Project/data/tiktok_travel_visa_dataset.csv
Output dir: /Users/richarddanquah/Desktop/Tuc/data analysis/Tiktok-Data-Analysis-Project/finalreport/summary_tables
Excel export: enabled


In [12]:
@dataclass
class Row:
    niche_category: str
    hook_style: str
    trending_sound_used: int
    video_duration_seconds: float
    views: int
    likes: int
    comments: int
    shares: int
    saves_or_favorites: int


def _to_int(value: str) -> int:
    s = (value or '').strip().replace(',', '')
    if not s:
        return 0
    return int(float(s))


def _to_float(value: str) -> float:
    s = (value or '').strip().replace(',', '')
    if not s:
        return 0.0
    return float(s)


def load_rows(csv_path: os.PathLike) -> List[Row]:
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        out: List[Row] = []
        for r in reader:
            out.append(
                Row(
                    niche_category=(r.get('niche_category') or '').strip(),
                    hook_style=(r.get('hook_style') or '').strip(),
                    trending_sound_used=_to_int(r.get('trending_sound_used') or '0'),
                    video_duration_seconds=_to_float(r.get('video_duration_seconds') or '0'),
                    views=_to_int(r.get('views') or '0'),
                    likes=_to_int(r.get('likes') or '0'),
                    comments=_to_int(r.get('comments') or '0'),
                    shares=_to_int(r.get('shares') or '0'),
                    saves_or_favorites=_to_int(r.get('saves_or_favorites') or '0'),
                )
            )
    return out


rows = load_rows(DATASET_CSV)
print('Loaded rows:', len(rows))
rows[0]

Loaded rows: 520


Row(niche_category='Travel Hacks', hook_style='Question', trending_sound_used=0, video_duration_seconds=11.23, views=5856, likes=195, comments=42, shares=36, saves_or_favorites=14)

## Helper functions
We compute engagement rate in two ways:
- **No saves**: `(likes + comments + shares) / views` (matches the formula used in the slides)
- **With saves**: `(likes + comments + shares + saves) / views` (if you want to include saves)


In [13]:
def er_no_saves(r: Row) -> float:
    if r.views <= 0:
        return 0.0
    return (r.likes + r.comments + r.shares) / r.views


def er_with_saves(r: Row) -> float:
    if r.views <= 0:
        return 0.0
    return (r.likes + r.comments + r.shares + r.saves_or_favorites) / r.views


def duration_band(seconds: float) -> str:
    if seconds < 15:
        return 'under_15'
    if seconds < 30:
        return '15_30'
    if seconds <= 45:
        return '30_45'
    if seconds <= 60:
        return '45_60'
    return 'over_60'


def mean(values: Iterable[float]) -> float:
    vals = list(values)
    return sum(vals) / len(vals) if vals else 0.0


def grouped_counts(rows: List[Row], key_fn) -> Dict[str, int]:
    out: Dict[str, int] = defaultdict(int)
    for r in rows:
        out[key_fn(r)] += 1
    return dict(out)


def grouped_means(rows: List[Row], group_key_fn, value_fn) -> Dict[str, float]:
    buckets: Dict[str, List[float]] = defaultdict(list)
    for r in rows:
        buckets[group_key_fn(r)].append(value_fn(r))
    return {k: mean(v) for k, v in buckets.items()}


## 1) Hook distribution
Counts and percentages of each `hook_style`.


In [14]:
hook_counts = grouped_counts(rows, lambda r: r.hook_style)
n = len(rows)
hook_table = [[k, hook_counts[k], round(hook_counts[k] / n * 100, 2)] for k in sorted(hook_counts)]
hook_table[:10]


[['Dramatic', 43, 8.27],
 ['Question', 146, 28.08],
 ['Statistic', 80, 15.38],
 ['Story', 104, 20.0],
 ['Tutorial', 147, 28.27]]

## 2) Engagement rate by hook type
Average engagement rate grouped by `hook_style`.


In [15]:
hook_er_no = grouped_means(rows, lambda r: r.hook_style, er_no_saves)
hook_er_w = grouped_means(rows, lambda r: r.hook_style, er_with_saves)
hook_er_table = [[k, round(hook_er_no.get(k, 0.0) * 100, 2), round(hook_er_w.get(k, 0.0) * 100, 2)]
                 for k in sorted(set(hook_er_no) | set(hook_er_w))]
hook_er_table


[['Dramatic', 4.39, 4.71],
 ['Question', 5.64, 6.01],
 ['Statistic', 5.39, 5.76],
 ['Story', 5.27, 5.6],
 ['Tutorial', 6.22, 6.67]]

## 3) Video duration vs performance
We bucket video length into bands and compute counts, average views, and average engagement rates.

Duration bands:
- under 15s
- 15–30s
- 30–45s
- 45–60s
- over 60s


In [16]:
band_counts = grouped_counts(rows, lambda r: duration_band(r.video_duration_seconds))
band_avg_views = grouped_means(rows, lambda r: duration_band(r.video_duration_seconds), lambda r: float(r.views))
band_avg_er_no = grouped_means(rows, lambda r: duration_band(r.video_duration_seconds), er_no_saves)
band_avg_er_w = grouped_means(rows, lambda r: duration_band(r.video_duration_seconds), er_with_saves)

band_order = ['under_15', '15_30', '30_45', '45_60', 'over_60']
duration_table = [[b, band_counts.get(b, 0), round(band_avg_views.get(b, 0.0), 0),
                   round(band_avg_er_no.get(b, 0.0) * 100, 2),
                   round(band_avg_er_w.get(b, 0.0) * 100, 2)]
                  for b in band_order]
duration_table


[['under_15', 66, 57550.0, 5.07, 5.38],
 ['15_30', 113, 116630.0, 5.65, 6.03],
 ['30_45', 206, 154582.0, 5.92, 6.34],
 ['45_60', 96, 89334.0, 5.36, 5.74],
 ['over_60', 39, 75914.0, 5.03, 5.35]]

## 4) Content category performance
Average engagement rate by `niche_category`.


In [17]:
cat_avg_er_no = grouped_means(rows, lambda r: r.niche_category, er_no_saves)
cat_avg_er_w = grouped_means(rows, lambda r: r.niche_category, er_with_saves)
category_table = [[c, round(cat_avg_er_no.get(c, 0.0) * 100, 2), round(cat_avg_er_w.get(c, 0.0) * 100, 2)]
                  for c in sorted(set(cat_avg_er_no) | set(cat_avg_er_w))]
category_table


[['Budget Travel', 5.53, 5.88],
 ['Destination Guides', 5.09, 5.44],
 ['Travel Hacks', 5.65, 6.02],
 ['Visa Application Tips', 5.93, 6.36]]

## 5) Trending sound visibility lift
We compare average views for videos with and without `trending_sound_used = 1`.


In [19]:
trend_views = [r.views for r in rows if r.trending_sound_used == 1]
nontrend_views = [r.views for r in rows if r.trending_sound_used == 0]
avg_trend = mean([float(v) for v in trend_views])
avg_non = mean([float(v) for v in nontrend_views])
lift = ((avg_trend / avg_non) - 1.0) * 100 if avg_non else 0.0
trending_table = [
    ['trending_sound_used=1', len(trend_views), round(avg_trend, 0)],
    ['trending_sound_used=0', len(nontrend_views), round(avg_non, 0)],
    ['lift_percent', '', round(lift, 2)],
]
trending_table


[['trending_sound_used=1', 210, 140559.0],
 ['trending_sound_used=0', 310, 99486.0],
 ['lift_percent', '', 41.29]]

## Export summary tables
This writes the summary tables to CSV. If `openpyxl` is installed, it will also write one Excel workbook with multiple sheets.


In [20]:
def write_csv(path: str, header: List[str], data: List[List[object]]) -> None:
    with open(path, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(header)
        w.writerows(data)


def write_xlsx(path: str, sheets: Dict[str, Tuple[List[str], List[List[object]]]]) -> None:
    if Workbook is None:
        raise RuntimeError('openpyxl is not installed. Install: python3 -m pip install openpyxl')
    wb = Workbook()
    first = True
    for name, (header, data) in sheets.items():
        if first:
            ws = wb.active
            ws.title = name
            first = False
        else:
            ws = wb.create_sheet(title=name)
        ws.append(header)
        for row in data:
            ws.append(row)
    wb.save(path)


write_csv(os.path.join(OUTPUT_DIR, 'hook_distribution.csv'), ['hook_style', 'count', 'percent'], hook_table)
write_csv(
    os.path.join(OUTPUT_DIR, 'engagement_by_hook.csv'),
    ['hook_style', 'avg_engagement_rate_no_saves_percent', 'avg_engagement_rate_with_saves_percent'],
    hook_er_table,
)
write_csv(
    os.path.join(OUTPUT_DIR, 'duration_band_performance.csv'),
    ['duration_band', 'count', 'avg_views', 'avg_engagement_rate_no_saves_percent', 'avg_engagement_rate_with_saves_percent'],
    duration_table,
)
write_csv(
    os.path.join(OUTPUT_DIR, 'category_engagement.csv'),
    ['niche_category', 'avg_engagement_rate_no_saves_percent', 'avg_engagement_rate_with_saves_percent'],
    category_table,
)
write_csv(os.path.join(OUTPUT_DIR, 'trending_sound_lift.csv'), ['group', 'count', 'avg_views'], trending_table)

if Workbook is not None:
    write_xlsx(
        OUTPUT_XLSX,
        {
            'hook_distribution': (['hook_style', 'count', 'percent'], hook_table),
            'engagement_by_hook': (['hook_style', 'avg_er_no_saves_percent', 'avg_er_with_saves_percent'], hook_er_table),
            'duration_band_perf': (['duration_band', 'count', 'avg_views', 'avg_er_no_saves_percent', 'avg_er_with_saves_percent'], duration_table),
            'category_engagement': (['niche_category', 'avg_engagement_rate_no_saves_percent', 'avg_engagement_rate_with_saves_percent'], category_table),
            'trending_sound_lift': (['group', 'count', 'avg_views'], trending_table),
        },
    )
    print('Wrote Excel workbook:', OUTPUT_XLSX)

print('Wrote CSV summary tables to:', OUTPUT_DIR)


Wrote Excel workbook: /Users/richarddanquah/Desktop/Tuc/data analysis/Tiktok-Data-Analysis-Project/finalreport/summary_tables/reproducible_summary_tables.xlsx
Wrote CSV summary tables to: /Users/richarddanquah/Desktop/Tuc/data analysis/Tiktok-Data-Analysis-Project/finalreport/summary_tables
